<a href="https://colab.research.google.com/github/blid11/Group9/blob/Brett/BrettG9_PartC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Xinwei' work


I have been created the 4 model - random forest, logistic regression, SVM, and Multilayer perception.
number of features depend on the cumulatife explained variance ratio (generally chossing number which >= 90%) 

In [ ]:
# Import libaries needed
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split # split data into training and testing sets
from sklearn.preprocessing import scale # scale and canter data
from sklearn.svm import SVC # make a svm for classification
from sklearn.model_selection import GridSearchCV # do cross validation
from sklearn.metrics import confusion_matrix # creates a confusion matrix
from sklearn.metrics import plot_confusion_matrix # draw confutionm matrix
from sklearn.decomposition import PCA # perform PAC to plot the data
import matplotlib.pyplot as plt
from matplotlib import style 
import numpy as np
import seaborn as sns


In [4]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

%cd gdrive/My Drive/G9A2/G9
 
#%cd gdrive/My Drive/G9A2
# I made a folder on my google drive that has the G9 Methylation data and the binfpy library
#this code automatically loads it in
# you can do the same with your google drive. 
# restart runtime if you experience errors!

Mounted at /content/gdrive
/content/gdrive/My Drive/G9A2/G9


In [ ]:
!ls

G9_liver_dna-meth.csv	mystery_dna-meth.csv
G9_liver_gene-expr.csv	mystery_gene-expr.csv


In [71]:
# load the data 
liver_gene = pd.read_csv('G9_liver_gene-expr.csv', index_col=0)
liver_methyl = pd.read_csv('G9_liver_dna-meth.csv', index_col=0)

mys_gene = pd.read_csv("mystery_gene-expr.csv", index_col=0) 
mys_methyl = pd.read_csv("mystery_dna-meth.csv",index_col=0)

In [ ]:
liver_gene.head(n=2)

,Label,X..100130426,X..100133144,X..100134869,X..10357,X..10431,X..136542,X..155060,X..26823,X..280660,...,ZXDA.7789,ZXDB.158586,ZXDC.79364,ZYG11A.440590,ZYG11B.79699,ZYX.7791,ZZEF1.23140,ZZZ3.26009,psiTPTE22.387590,tAKR.389932
TCGA.2V.A95S.01,Primary Tumor,0.0,1.5051,3.7074,90.1124,1017.1038,0.0,141.3911,0.6516,0.0,...,24.7597,273.6602,794.2662,18.2440,499.1041,3172.5037,890.0472,510.1808,3.9094,6.5157
TCGA.2Y.A9GS.01,Primary Tumor,0.0,26.4120,2.6663,71.0054,639.2311,0.0,122.7206,1.4786,0.0,...,68.5067,632.8241,1153.7703,71.4638,1000.4929,5301.1336,755.5446,860.5224,6.4071,482.9966


In [ ]:
liver_methyl.head(n=2)

## 1. Data Preparation



### 1.1 Data Cleaning
- Rows without labels should be removed. 
- Separate Data and Labels
- NA values may need to be replaced for impute functions to work.

#### Separate Data and Labels

In [72]:
# Data for liver datasets
liver_gene_data = liver_gene[liver_gene.columns[2:]]
liver_methyl_data = liver_methyl[liver_methyl.columns[2:]]
# For mystery datasets
mys_gene_data = mys_gene[mys_gene.columns[2:]]
mys_methyl_data = mys_methyl[mys_methyl.columns[2:]]

# Labels for liver datasets
liver_gene_labels = [1 if value == 'Primary Tumor' else 0 for value in liver_gene['Label'].values]
liver_methyl_labels = [1 if value == 'Primary Tumor' else 0 for value in liver_methyl['Label'].values]
# Labels for mystery datasets
mys_methyl_labels = [1 if value == 'Primary Tumor' else 0 for value in mys_gene['Label'].values]
mys_gene_labels = [1 if value == 'Primary Tumor' else 0 for value in mys_methyl['Label'].values]


## 2. Pre-processing Data

### Log base 2 of data and log fold change.

In [73]:
for col_name in liver_gene_data.columns:
  #IMPUTE data
  col_data = liver_gene_data[col_name].fillna(liver_gene_data[col_name].mean(skipna= True))
  #take log base 2
  col_data = pd.to_numeric(np.log2(col_data.values + 1))
  liver_gene_data[col_name] = col_data

lg_logs_solid_tum = pd.DataFrame()
lg_logs_primary_tum = pd.DataFrame()

index = 0 
for label in liver_gene_labels:
  if label: 
    lg_logs_solid_tum = pd.concat([lg_logs_solid_tum, liver_gene_data.iloc[[index]]], axis= 0)
  else:
    lg_logs_primary_tum = pd.concat([lg_logs_primary_tum, liver_gene_data.iloc[[index]]], axis = 0)
  index += 1

lg_log_fcs = {}

for col_name in lg_logs_solid_tum.columns:
  fc_solid = lg_logs_solid_tum[col_name].mean()
  fc_prim = lg_logs_primary_tum[col_name].mean()
  log2fc = abs(fc_solid - fc_prim)
  lg_log_fcs[col_name] = log2fc

keep_genes = []
for gene in lg_log_fcs.keys():
  if lg_log_fcs[gene] > 1: 
    keep_genes.append(gene)

liver_gene_data2 = liver_gene_data[keep_genes]
print(liver_gene_data2.shape)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(421, 3696)


In [65]:
liver_gene_data2.head()

,X..155060,X..653553,A1BG.1,A2M.2,AAA1.404744,AADAC.13,AADAT.51166,AASS.10157,ABAT.18,ABCA13.154664,...,ZNF85.7639,ZNF862.643641,ZNF883.169834,ZP3.7784,ZPLD1.131368,ZSCAN16.80345,ZSWIM5.57643,ZWINT.11130,psiTPTE22.387590,tAKR.389932
TCGA.2V.A95S.01,7.153715,9.630558,14.443673,18.127270,0.723864,10.414410,4.687044,5.601836,10.584448,0.000000,...,4.449581,8.080822,2.295547,6.792106,4.266337,7.072028,8.118516,8.858813,2.295547,2.909907
TCGA.2Y.A9GS.01,6.950942,9.070705,14.466773,14.927845,0.578118,13.339532,7.292245,8.815327,13.453890,4.265707,...,6.831162,7.953729,5.455436,5.110760,7.662327,6.707145,9.772858,9.298936,2.888909,8.918853
TCGA.2Y.A9GT.01,6.585654,6.714419,16.245087,14.252355,1.883621,12.764681,8.906339,11.557786,13.618564,4.706337,...,4.603668,8.389972,1.481971,3.939847,0.000000,6.303626,3.181087,8.165080,3.939847,6.398990
TCGA.2Y.A9GU.01,8.135816,8.393708,13.186655,17.896176,0.000000,12.727613,5.204297,10.440706,13.239476,2.116564,...,3.088701,9.666433,1.415921,4.870069,0.875072,6.133576,6.008868,8.818376,1.808344,1.808344
TCGA.2Y.A9GV.01,8.147571,7.605401,16.362278,13.096133,3.241871,12.794975,9.268905,9.447338,14.493701,3.775093,...,4.020449,8.629696,2.132281,5.608818,0.000000,6.899364,1.428678,6.655793,1.822975,6.910365


In [74]:
for col_name in liver_methyl_data.columns:
  #IMPUTE data
  col_data = liver_methyl_data[col_name].fillna(liver_methyl_data[col_name].mean(skipna= True))
  #take log base 2
  col_data = pd.to_numeric(np.log2(col_data.values + 1))
  liver_methyl_data[col_name] = col_data

lm_logs_solid_tum = pd.DataFrame()
lm_logs_primary_tum = pd.DataFrame()

index = 0 
for label in liver_methyl_labels:
  if label: 
    lm_logs_solid_tum = pd.concat([lm_logs_solid_tum, liver_methyl_data.iloc[[index]]], axis= 0)
  else:
    lm_logs_primary_tum = pd.concat([lm_logs_primary_tum, liver_methyl_data.iloc[[index]]], axis= 0)
  index += 1

lm_log_fcs = {}

for col_name in lm_logs_solid_tum.columns:
  fc_solid = lm_logs_solid_tum[col_name].mean()
  fc_prim = lm_logs_primary_tum[col_name].mean()
  log2fc = abs(fc_solid - fc_prim)
  lm_log_fcs[col_name] = log2fc

keep_tags = []
for tag in lm_log_fcs.keys():
  if lm_log_fcs[tag] > 0.3: 
    keep_tags.append(tag)

liver_methyl_data2 = liver_methyl_data[keep_tags]
print(liver_methyl_data2.shape)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(427, 2806)


In [ ]:
mys_gene_data = mys_gene_data[keep_genes]
mys_methyl_data = mys_methyl_data[keep_tags]

for col_name in mys_gene_data.columns:
  col_data = mys_gene_data[col_name].fillna(mys_gene_data[col_name].mean(skipna= True))
  #take log base 2
  col_data = pd.to_numeric(np.log2(col_data.values + 1))
  mys_gene_data[col_name] = col_data

for col_name in mys_methyl_data.columns:
  col_data = mys_methyl_data[col_name].fillna(mys_methyl_data[col_name].mean(skipna= True))
  #take log base 2
  col_data = pd.to_numeric(np.log2(col_data.values + 1))
  mys_methyl_data[col_name] = col_data

#### Scaling Data

In [ ]:
# apply minmaxscaler to X
from sklearn.preprocessing import MinMaxScaler,LabelEncoder

liver_gene_data2 = MinMaxScaler().fit_transform(liver_gene_data2)
print
print(liver_gene_data2.head(n=2))
liver_methyl_data2 = MinMaxScaler().fit_transform(liver_methyl_data2)
print(liver_methyl_data2.head(n=2))
mys_gene_data = MinMaxScaler().fit_transform(mys_gene_data)
print(mys_gene_data.head(n=2))
mys_methyl_data = MinMaxScaler().fit_transform(mys_methyl_data)
print(mys_methyl_data.head(n=2))

### Feature Selection: 
-Want to use only a subset of genes/methylation tags
for each model

#### Removing Genes with Low Expression

In [ ]:
expr_means = []

for column in liver_gene_expr[col_names_exp[2:]]:
  col_mean = liver_gene_expr[column].mean()
  expr_means.append(col_mean)
  if col_mean > 0.025:
    liver_gene_expr[column] = liver_gene_expr[column].replace([np.inf, np.nan, -np.inf], col_mean)
  else: 
    liver_gene_expr = liver_gene_expr.drop(column, axis = 1)

print(liver_gene_expr.shape)

plt.hist(expr_means, bins = 40, range = [0.0, 0.2])
plt.show()